In [1]:
import random
from numpy.random import choice
import numpy as np
import matplotlib.pyplot as plt
 

#those are our target function we aim to find the minimum solutions for them===============
#  Bent cigar function (input:1D list)
def bent_cigar_func(trail_vector,D):
    result = pow(trail_vector[0],2)
   
    for i in range(1,D):    
        result+=pow(10,6)*pow(trail_vector[i],2)
    return result

# Discus function
def discus_func(trail_vector,D):
    result = pow(trail_vector[0],2)*pow(10,2)
    for i in range(1,D):
        result+=pow(trail_vector[i],2)
    return result

# High Conditioned Elliptic function
def HCE_func(trail_vector,D):
    result = 0
    for i in range(D):
        result+=pow(10,6*(i/4))*pow(trail_vector[i],2)
    return result

# Rosenbrock's function
def rosenbrock_func(trail_vector,D):
    result=0
    for i in range(D-1):
        result+=100*pow( pow(trail_vector[i],2)-trail_vector[i+1], 2 )+pow(trail_vector[i]-1,2)
    return result
#==========================================================================================
#==========================================================================================
    
# the initialization function to generate initial random populations
def initialization(NP, D):
    population=[]
    for i in range(NP):
        population.append([])
        for j in range(D):
            #population[i].append(random.randint(-10,10))
            population[i].append(random.uniform(-10,10))
    
    #print("initial population :",population)
    return population
 
#calculate fitness value for bounded trial vectors(input:2D list)
def calcFitness(population,func,D):
    score = []
    
    for i in range(len(population)):
        #print("fitness:",func(population[i]))
        if func == "Bent":
            score.append(bent_cigar_func(population[i],D))
        elif func == "Discus":
            score.append(discus_func(population[i],D))
        elif func == "HCE":
            score.append(HCE_func(population[i],D))
        elif func == "Rosenbrock":
            score.append(rosenbrock_func(population[i],D))
    return score

def mutation(population, targetIndex,F=0.8):
    donor_vector=[]
    #randomly choose from the population matrix
    picklist=[i for i in range(0, len(population))]
    picklist.remove(targetIndex) #to prevent select same index as the target vector, we remove the target vector index
    #print(picklist)
    
    temp=np.random.choice(picklist,3,replace=False)
    first_vector = population[temp[0]]
    #print("V1:",first_vector)
    
    second_vector = population[temp[1]]
    #print("V2:",second_vector)
    
    third_vector = population[temp[2]]
    #print("V3:",third_vector)
    for i in range(len(first_vector)):
        donor_vector.append(first_vector[i]+F*(second_vector[i]-third_vector[i]))
    return donor_vector

# Binomial crossover to determine the trail_vector
def crossover(donor_vector,target_vector,Cr=0.9):
    trial_vector=[]
    r_random=[]
    for j in range(len(target_vector)):
        r_trial=random.uniform(0,1)
        r_random.append(r_trial)
       
    picklist=[i for i in range(0, len(donor_vector))]
    location = random.choice(picklist)
    
    for k in range(len(donor_vector)):
        if r_random[k] <= Cr or k == location:
            trial_vector.append(donor_vector[k])
        elif r_random[k] > Cr and k != location:
            trial_vector.append(target_vector[k])
  
    return trial_vector
 
# Adjust bounds of offspring
def bound(x):
    for i in range(len(x)):
        if x[i] <= 10 and x[i] >= -10:
            pass
        elif x[i] > 10:
            x[i]= 10
        elif x[i] < -10:
            x[i]= -10
    return x

# Greedy select population function
def select(next_g,population,current_score,next_score):
    solution=[]
    
    for i in range(len(population)):
        if next_score[i] <= current_score[i]:
            solution.append(next_g[i])
         
        else:
            solution.append(population[i])
          
    return solution


    
# In[2]:
#######main process Here#############
def sovler(func_name):
# NP=100 number of population at first generation
# D=5    dimension
#Cr=0.9  Crossover factor
#F=0.8   Mutation factor

#MAXFES= 5000*D the maximum number of generations
    D=5 #dimension
    NP=100
    Cr=0.9
    F=0.8
    MAXFES=D*5000#25000
    donor_vector = []
    target_vector = []
    trail_vector = []
    bounded_vector = []
    population = []
    population = initialization(NP,D)
    next_score = []
    solution=[]
   
    
    # start the process
    for i in range(MAXFES):
    #print("Round:",i) 
    
        #the fitness score for this population
        current_score = calcFitness(population,func_name,D) #[45000049, 10000004, 110000009, 115000016, 115000049]

        #print("Current_score :",current_score)
        #print("current_population",population)
        next_g=[]# this will save the vector for next generation
        m=0 # the index count for selected vector
        
        #iterate through all the vectors in the population
        while m<NP :
            #select target vector
            target_vector= population[m]
             # Mutate the population
            donor_vector=mutation(population,m)
            # Compare each element in both vector and determine the survive individual for the next generation
            trail_vector=crossover(donor_vector,target_vector)
            # Control the value of X to ensure its within [-10,10]
            bounded_vector=bound(trail_vector) #bounded population U

            next_g.append(bounded_vector)
        #print("target_v:",target_vector)
        #print("donor_v:",donor_vector)
        #print("trail_v:",trail_vector)
            m+=1
            
            
        # Get a fitness score for the modified population (next generation)
        #print("Next_g",next_g) 
        next_score = calcFitness(next_g,func_name,D) #[159000100, 114000016, 198000081, 187000081, 98000025]    

        population=select(next_g,population,current_score,next_score)
    #print("new_population",population)
        
    minvalue = min(next_score)
    solution = population[next_score.index(minvalue)]
    return minvalue,solution
    #print("F(x)min:",minvalue)
    #print("Solution:",population[next_score.index(minvalue)])
solution_1=[]
min_1,solution_1= sovler("Bent")
print("Minimum of function:",min_1)
print("Solution:",solution_1)

solution_2=[]
min_2,solution_2= sovler("Discus")
print("Minimum of function:",min_2)
print("Solution:",solution_2)

solution_3=[]
min_3,solution_3= sovler("HCE")
print("Minimum of function:",min_3)
print("Solution:",solution_3)

solution_4=[]
min_4,solution_4= sovler("Rosenbrock")
print("Minimum of function:",min_4)
print("Solution:",solution_4)


Minimum of function: 0.0
Solution: [5.128700802147307e-164, -1.5092191858438485e-162, 1.0869523116216768e-162, -1.0475572743565441e-163, -5.843867474770704e-163]
Minimum of function: 0.0
Solution: [-2.319561895890816e-163, 1.2942755626289218e-162, 5.5235344648359526e-163, -8.041852431890444e-163, 7.375894118866588e-163]
Minimum of function: 0.0
Solution: [-4.455772232034743e-163, -1.4975666322370007e-162, -4.8418716665199775e-163, 7.018894032358936e-163, -7.411611119102132e-163]
Minimum of function: 0.0
Solution: [1.0, 1.0, 1.0, 1.0, 1.0]
